# Mark-ICP tester

In [1]:
import os
import sys
import cv2
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/milestone_202110'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE

from demo_utils.environment import add_cam, add_bed, move_bed

from utils.detection_util import MultiICP, MODEL_DIR, ColorDepthMap

# from util.multiICP import *
# from test_util import *

In [2]:
crob = CombinedRobot(robots_on_scene=[RobotConfig(0, RobotType.indy7, 
                           ((0,0,0), (0,0,0)),None)]
              , connection_list=[False])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
viewpoint = add_cam(gscene, tool_link="indy0_tcp")
VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
gscene.show_pose(VIEW_POSE)
Tbc = viewpoint.get_tf(VIEW_POSE)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Get ICP result

In [3]:
os.getcwd()

'/home/jhkim/Projects/rnb-planning/src/scripts/perception'

In [4]:
color_image = cv2.imread(os.path.join(os.getcwd(), "save_img/bed/color/color_0.jpg"), flags=cv2.IMREAD_UNCHANGED)
depth_mask = cv2.imread(os.path.join(os.getcwd(), "save_img/bed/depth_segmented/depth_mask_0.png"), flags=cv2.IMREAD_UNCHANGED)

cdp = ColorDepthMap(color_image, depth_mask, 
                    [1280, 720,
                     899.05322266, 899.21044922,
                     654.88366699, 352.92956543], 
                    0.000250000011874)
cameraMatrix = np.array([[cdp.intrins[2], 0, cdp.intrins[4]],
                         [0, cdp.intrins[3], cdp.intrins[5]],
                         [0, 0, 1]])
distCoeffs = np.array([ 0.15870179, -0.46903715, -0.0014261 ,  0.00066797,  0.41631824])

icp_bed = MultiICP(model=MODEL_DIR + '/bed/bed.STL', 
                   Toff=SE3([[0,1,0],[0,0,1],[1,0,0]], [0.455,0,1.02]))

icp_bed.clear()
pcd = icp_bed.add_image(cdp, Tc=None)


### Show on RVIZ

In [5]:
gscene.create_safe(GEOTYPE.MESH, "bed_pcd", "base_link", (0.1,0.1,0.1), (0,0,0), rpy=(0,0,0), color=(1,0,0,1), display=True,
                   collision=False, fixed=True, scale=(0.01,0.01,1),
                   vertices=np.matmul(np.asarray(pcd.points), Tbc[:3,:3].transpose())+Tbc[:3,3])

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.


### Compute ICP

In [6]:
T_cb = SE3_inv(Tbc)
Tguess = icp_bed.get_initial_by_center(R=np.matmul(T_cb[:3,:3], Rot_axis(3,np.pi)), 
                                               offset=np.matmul(T_cb[:3,:3], (1.1*0.7,0,-0.6)))


# ICP_result, fitness = icp_bed.compute_ICP(To=SE3(Rot_axis_series([3, 2],[-np.pi/2, np.pi/2]), (0,0,3)), visualize=True)
T_co1, fitness1 = icp_bed.compute_ICP(To=Tguess, thres=0.1, visualize=True)
T_co2, fitness2 = icp_bed.compute_front_ICP(Tc_cur=np.identity(4), To=T_co1, thres=0.1, visualize=True)
Tdetect=np.matmul(Tbc, T_co2)

Apply point-to-point ICP
registration::RegistrationResult with fitness=4.863729e-01, inlier_rmse=4.163255e-02, and correspondence_set size of 16543
Access transformation to get result.
Transformation is:
[[ 0.90320448 -0.06870464  0.42367598 -0.49543093]
 [-0.1909966  -0.94832181  0.25338912  0.14790305]
 [ 0.38437216 -0.30978286 -0.86965086  4.35771749]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=5.247589e-01, inlier_rmse=3.987199e-02, and correspondence_set size of 8923
Access transformation to get result.
Transformation is:
[[ 0.90677202 -0.06157974  0.41710004 -0.48067304]
 [-0.16154115 -0.96452674  0.20878848  0.22888415]
 [ 0.389447   -0.25670237 -0.88455352  4.35373226]
 [ 0.          0.          0.          1.        ]]


### Visualize and adjust
* Adjust Tdetect to get perfect GT

In [7]:
bed = add_bed(gscene, Tdetect[:3,3], Rot2rpy(Tdetect[:3,:3]), (0,1,0,0.5))

In [23]:
T_bo_gt = matmul_series(Tdetect, 
                        SE3(np.identity(3), (0.015,0.025,0.05)),
                        SE3(Rot_axis(1, -0.0), (0,0.0,0)),
                        SE3(Rot_axis(2, 0.04), (0,0.0,0)),
                        SE3(Rot_axis(3, 0.01), (0,0.0,0))
                       )

In [24]:
move_bed(gscene, bed_center=T_bo_gt[:3,3], 
         bed_rpy=Rot2rpy(T_bo_gt[:3,:3]))

# Make ArucoMap for Testing

### Testing single marker
* Make single-marker aruco map to get one-marker transformation

In [25]:
from cv2 import aruco
from pkg.detector.aruco.detector import *

def get_aruco_map_test(test_name, mk_idx, mk_size, point=[0,0,0], direction=(0,0,0)):
    dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
    #     params = aruco.DetectorParameters_create()

    aruco_map = ArucoMap(dictionary=dictionary, _dict={
        test_name: MarkerSet(test_name,
                         dlevel=DetectionLevel.ENVIRONMENT, gtype=GEOTYPE.BOX, dims=(0.1, 0.1,0.1), color=(0.8,0.0,0.0,1),
                         _list=[
                             ObjectMarker(test_name, mk_idx, mk_size, point, direction)
                         ])
    })
    return aruco_map

In [26]:
mk_idx = 230
mk_size = 0.15
aruco_map = get_aruco_map_test("bed_test", mk_idx, mk_size)
objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
Tbmark = objectPose_dict['bed_test']
Tbmark = np.matmul(Tbc, Tbmark)

### Show marker location on RVIZ

In [27]:
gscene.add_highlight_axis("hl", "mark_{}".format(mk_idx), T=Tbmark)

### Get relative marker offset

In [28]:
Tmark = np.matmul(SE3_inv(Tdetect), Tbmark)

point = Tmark[:3,3]
direction = Rot2rpy(Tmark[:3,:3])
print("point: {}, {}, {}".format(*point))
print("direction: {}, {}, {}".format(*direction))

point: 1.15971701191, -0.321270790663, 0.14664528474
direction: -1.59720036358, 0.0221074130319, 1.59913372711


### Visualize result

In [14]:
aruco_map = get_aruco_map_test("bed_test", mk_idx, mk_size, point=point, direction=direction)
objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
Tbed = objectPose_dict['bed_test']
Tbed = np.matmul(Tbc, Tbed)

In [15]:
add_bed(gscene, Tbed[:3,3], Rot2rpy(Tbed[:3,:3]), (0,1,0,0.5))